In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import gc
from scipy.spatial.distance import pdist, squareform

In [93]:
prop_data = pd.read_csv("./data/properties_2017.csv")
# prop_data

In [94]:
train_data = pd.read_csv("./data/train_2017.csv", parse_dates=["transactiondate"])
# train_data

In [ ]:
# missing_val = prop_data.isnull().sum().reset_index()
# missing_val.columns = ['column_name', 'missing_count']
# missing_val = missing_val.loc[missing_val['missing_count']>0]
# missing_val = missing_val.sort_values(by='missing_count')

# missing_val['missing_ratio'] = missing_val["missing_count"]/prop_data.shape[0]
# missing_val = missing_val.loc[missing_val["missing_ratio"]>0.6]
# missing_val


# ind = np.arange(missing_val.shape[0])
# width = 0.9
# fig, ax = plt.subplots(figsize=(12,18))
# rects = ax.barh(ind, missing_val.missing_ratio.values, color='blue')
# ax.set_yticks(ind)
# ax.set_yticklabels(missing_val.column_name.values, rotation='horizontal')
# ax.set_xlabel("Count of missing values")
# ax.set_title("Number of missing values in each column")
# plt.show()

# del ind


In [ ]:
# prop_data.drop(missing_val.column_name.values, axis=1, inplace=True)

In [ ]:
# prop_data

In [ ]:
# prop_data_temp = prop_data.fillna(prop_data.mean(), )
plt.plot(prop_data.groupby("regionidcounty")["taxvaluedollarcnt"].mean())
plt.show()

In [95]:
original = prop_data.copy()

In [29]:
prop_data = original.copy()

In [96]:
# prop_data['actual_area'] = prop_data[['finishedfloor1squarefeet','calculatedfinishedsquarefeet','finishedsquarefeet12', 'finishedsquarefeet13',
#                       'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6']].max(axis=1)


prop_data.loc[(prop_data["heatingorsystemtypeid"]==2.0) & (pd.isnull(prop_data["airconditioningtypeid"])), "airconditioningtypeid"] = 1.0

prop_data["airconditioningtypeid"].fillna(0, inplace=True)
prop_data["airconditioningtypeid"] = prop_data["airconditioningtypeid"].map({1:1,13:1,5:0,9:1,11:1,12:1,3:1})
# prop_data['airconditioningtypeid'].value_counts()

prop_data['actual_area'] = prop_data['calculatedfinishedsquarefeet']#.value_counts(dropna = False)
prop_data['calculatedbathnbr'].fillna(prop_data['calculatedbathnbr'].median(),inplace = True)
prop_data['bedroomcnt'].fillna(prop_data['bedroomcnt'].median(), inplace = True)

prop_data['taxvaluedollarcnt'].fillna(prop_data["taxvaluedollarcnt"].mean(), inplace=True)

prop_data['actual_area'].replace(to_replace=1.0,value=np.nan,inplace=True)
prop_data['actual_area'].fillna(prop_data['actual_area'].median(),inplace=True)

prop_data['unitcnt'].fillna(1, inplace = True)

prop_data['latitude'].fillna(prop_data['latitude'].median(),inplace = True)
prop_data['longitude'].fillna(prop_data['longitude'].median(),inplace = True)

prop_data['lotsizesquarefeet'].fillna(prop_data['lotsizesquarefeet'].median(), inplace = True)

prop_data["poolcnt"].fillna(0, inplace=True)
prop_data["fireplacecnt"].fillna(0, inplace=True)
prop_data["hashottuborspa"].fillna(0, inplace=True)
prop_data['hashottuborspa'] = pd.to_numeric(prop_data['hashottuborspa'])

prop_data["taxdelinquencyflag"].fillna(-1, inplace=True)
prop_data["taxdelinquencyflag"] = prop_data["taxdelinquencyflag"].map({'Y':1, -1:-1})

prop_data.loc[(prop_data["heatingorsystemtypeid"]==2.0) & (pd.isnull(prop_data["airconditioningtypeid"])), "airconditioningtypeid"] = 1.0
prop_data["airconditioningtypeid"].fillna(-1, inplace=True)

prop_data["buildingqualitytypeid"].fillna(7, inplace=True)

prop_data["yearbuilt"].fillna(prop_data["yearbuilt"].mean(), inplace=True)
prop_data["age"] = 2017 - prop_data["yearbuilt"]
#imputing garagecarcnt on basis of propertylandusetypeid
#All the residential places have 1 or 2 garagecarcnt, hence using random filling for those values.

prop_data.loc[(prop_data["propertylandusetypeid"]==261) & (pd.isnull(prop_data["garagecarcnt"])), "garagecarcnt"] = np.random.randint(1,3)
prop_data.loc[(prop_data["propertylandusetypeid"]==266) & (pd.isnull(prop_data["garagecarcnt"])), "garagecarcnt"] = np.random.randint(1,3)
prop_data["garagecarcnt"].fillna(0, inplace=True)

prop_data["taxamount"].fillna(prop_data.taxamount.mean(), inplace=True)

#### Remove the abs part for longtitude, perform on whatever df requires it
# prop_data['longitude'] = prop_data['longitude'].abs()

In [ ]:
# prop_data['calculatedfinishedsquarefeet'].describe()

### Normalizing the data

In [31]:

colsList = ["actual_area",
            "poolcnt",
            "latitude",
            "longitude",
            "unitcnt",
            "lotsizesquarefeet",
            "bedroomcnt",
            "calculatedbathnbr",
            "hashottuborspa",
            "fireplacecnt",
            "taxvaluedollarcnt",
            "buildingqualitytypeid",
            "garagecarcnt",
            "age",
            "taxamount"]

In [ ]:
prop_data_ahp = prop_data[colsList]
# prop_data_ahp

In [ ]:
for col in prop_data_ahp.columns:
    prop_data_ahp[col] = (prop_data_ahp[col] - prop_data_ahp[col].mean())/prop_data_ahp[col].std(ddof=0)

In [ ]:
# prop_data_ahp.isnull().sum()

In [ ]:
# for cols in prop_data_ahp.columns.values:
#     print prop_data_ahp[cols].value_counts(dropna=False)


## Analytical Hierarchical Processing

In [ ]:
rel_imp_matrix = pd.read_csv("./data/rel_imp_matrix.csv", index_col=0)
# rel_imp_matrix

In [ ]:
import fractions


for col in rel_imp_matrix.columns.values:
    temp_list = rel_imp_matrix[col].tolist()
    rel_imp_matrix[col] = [float(fractions.Fraction(x))  for x in temp_list]
    
#     data = [float(fractions.Fraction(x)) for x in data]
# rel_imp_matrix


In [ ]:
for col in rel_imp_matrix.columns.values:
    rel_imp_matrix[col] /= rel_imp_matrix[col].sum()
    
# rel_imp_matrix

In [ ]:
rel_imp_matrix["row_sum"] = rel_imp_matrix.sum(axis=1)

rel_imp_matrix["score"] = rel_imp_matrix["row_sum"]/rel_imp_matrix.shape[0]

rel_imp_matrix.to_csv("./data/final_score_matrix.csv", index=False)
# rel_imp_matrix

In [ ]:
ahp_column_score = rel_imp_matrix["score"]

In [ ]:
# prop_data_ahp.info()

In [ ]:
# prop_data_ahp.drop('sum', axis=1,inplace=True)
# prop_data_ahp.keys()

# SAW

In [ ]:
sum_series = pd.Series(0, index=prop_data_ahp.index,dtype='float32')

for col in prop_data_ahp.columns:
    sum_series = sum_series+ prop_data_ahp[col] * ahp_column_score[col]
prop_data_ahp["sum"] = sum_series.astype('float32')

In [ ]:
# prop_data_ahp["sum"]
# prop_data_ahp["sum"] = prop_data_ahp.sum(axis=1)
    

In [ ]:
prop_data_ahp["sum"].describe()

In [ ]:
prop_data_ahp.sort_values(by='sum', inplace=True)

In [ ]:
prop_data_ahp.head(n=10)

In [ ]:
prop_data_ahp.tail(n=10)

In [ ]:
print prop_data[colsList].iloc[1252741],"\n\n"
print prop_data[colsList].iloc[342941]

In [101]:
distance_cols  = ["actual_area",
                "poolcnt",
                "latitude",
                "longitude",
                "unitcnt",
                "lotsizesquarefeet",
                "bedroomcnt",
                "calculatedbathnbr",
                "hashottuborspa",
                "fireplacecnt",
                "taxvaluedollarcnt",
                "buildingqualitytypeid",
                "garagecarcnt",
                "age",
                "taxamount",
                "airconditioningtypeid",
                 "regionidcounty"]

In [60]:
col_score = pd.read_csv('./data/column_scores_distance_metric.csv',index_col=0)

In [ ]:
# # col_score = col_score['score']
# for i in distance_cols:
#     print i,"  ",col_score.ix[i]['score']

In [57]:
train_data.drop_duplicates('parcelid',inplace=True)

prop_data_dist = pd.merge(prop_data,train_data,on='parcelid')
prop_data_dist = prop_data_dist[distance_cols]
prop_data_dist.fillna(3101,inplace=True)
prop_data_dist['regionidcounty'] = prop_data_dist['regionidcounty'].map({3101:1,1286:0,2061:-1}) 

In [ ]:

# prop_data_dist['regionidcounty'].value_counts(dropna=False)

In [58]:
for col in prop_data_dist.columns:
    prop_data_dist[col] = (prop_data_dist[col] - prop_data_dist[col].mean())/prop_data_dist[col].std(ddof=0)
# prop_data_dist['airconditioningtypeid'] = (prop_data['airconditioningtypeid'] - prop_data['airconditioningtypeid'].mean())/prop_data['airconditioningtypeid'].std(ddof=0)

In [61]:
for i in distance_cols:
    if i == 'regionidcounty':
        prop_data_dist[i] = prop_data_dist[i]*col_score.ix['samecounty']['score']
        continue
    prop_data_dist[i] = prop_data_dist[i]*col_score.ix[i]['score']

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [2]:
prop_data_dist.to_csv('prop_data_dist.csv',index=False)

NameError: name 'prop_data_dist' is not defined

In [40]:
prop_data_dist = pd.read_csv("prop_data_dist.csv")

# for i in prop_data_dist.columns:
#     prop_data_dist[i] = prop_data_dist[i].astype("float32")
#     print prop_data_dist[i].dtype

In [63]:
# from sklearn.cluster import KMeans

# kmeans = KMeans( verbose=1, n_jobs=-1).fit(prop_data_dist)
# kmeans.labels_

result = pdist(prop_data_dist[:20000],metric='cityblock')

In [42]:
class_6 = []
for i in range(len(kmeans.labels_)):
    class_6.append(kmeans.labels_[i] == 7)
# prop_data_dist[class_6]

In [64]:
result = squareform(result)

In [ ]:
temp = pd.merge(prop_data,train_data,on='parcelid')

temp = temp[distance_cols]

In [68]:
mini, minj = 0,1
for i in range(result.shape[0]):
    for j in range(i+1, result.shape[1]):
        if result[i, j] < result[mini, minj] and result[i, j] != 0:
            mini, minj  = i, j
            

In [73]:
print mini, minj, result[mini, minj]

3136 13926 2.61775692388e-07


In [ ]:
print temp.iloc[mini]

print temp.iloc[minj]

In [74]:
maxi, maxj = 0,1
for i in range(result.shape[0]):
    for j in range(i+1, result.shape[1]):
        if result[i, j] > result[maxi, maxj]:
            maxi, maxj  = i, j

In [75]:
print maxi, maxj, result[maxi, maxj]

7070 17158 49.0194472225


In [76]:
print temp.iloc[maxi]

print temp.iloc[maxj]

actual_area              8.790000e+03
poolcnt                  1.000000e+00
latitude                 3.408038e+07
longitude                1.184174e+08
unitcnt                  1.000000e+00
lotsizesquarefeet        4.849700e+04
bedroomcnt               7.000000e+00
calculatedbathnbr        5.000000e+00
hashottuborspa           0.000000e+00
fireplacecnt             0.000000e+00
taxvaluedollarcnt        2.385837e+07
buildingqualitytypeid    1.100000e+01
garagecarcnt             1.000000e+00
age                      9.100000e+01
taxamount                2.767978e+05
airconditioningtypeid    1.000000e+00
regionidcounty           3.101000e+03
Name: 7070, dtype: float64
actual_area              3.472000e+03
poolcnt                  0.000000e+00
latitude                 3.371964e+07
longitude                1.180409e+08
unitcnt                  2.370000e+02
lotsizesquarefeet        6.314000e+03
bedroomcnt               0.000000e+00
calculatedbathnbr        2.000000e+00
hashottuborspa         

## Distance Metric

We will be using weighted Manhattan distance as a distance metric

## Implementing Neural Network

In [10]:
from sklearn.preprocessing import LabelEncoder
import datetime as dt
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer


print( "\n\nProcessing data for Neural Network ...")
print('\nLoading train, prop and sample data...')
# train_data = pd.read_csv("./train_2016_v2.csv", parse_dates=["transactiondate"])
train_data
# prop = pd.read_csv('../input/properties_2016.csv')

prop = prop_data#pd.read_csv("./train_data_clean_2.csv")
sample = pd.read_csv('./sample_submission.csv')
 
print('Fitting Label Encoder on properties...')
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))
        
print('Creating training set...')
df_train = train_data.merge(prop, how='left', on='parcelid')

df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train["transactiondate_year"] = df_train["transactiondate"].dt.year
df_train["transactiondate_month"] = df_train["transactiondate"].dt.month
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
df_train["transactiondate"] = df_train["transactiondate"].dt.day

# print('Creating x_train and y_train from df_train...' )
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate'], axis=1)
y_train = df_train["logerror"]

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

    
print('Creating df_test...')
sample['parcelid'] = sample['ParcelId']
test_months = [k for k in sample.columns.values if k not in ["parcelid"]]

print("Merging Sample with property data...")
df_test = sample.merge(prop, on='parcelid', how='left')


# ## Preprocessing
print("\nPreprocessing neural network data...")
# imputer= Imputer()
# imputer.fit(x_train.iloc[:, :])
# x_train = imputer.transform(x_train.iloc[:, :])
# imputer.fit(x_test.iloc[:, :])
# x_test = imputer.transform(x_test.iloc[:, :])

x_test.fillna(x_test.mean(), inplace=True)


# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)
# x_test = sc.transform(x_test)

len_x=int(x_train.shape[1])
print("len_x is:",len_x)


# Neural Network
print("\nSetting up neural network model...")
nn = Sequential()
nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.63))
nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.45))
nn.add(Dense(units = 28, kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(1, kernel_initializer='normal'))
nn.compile(loss='mae', optimizer=Adam(lr=4e-3, decay=1e-4))

print("\nFitting neural network model...")
nn.fit(np.array(x_train), np.array(y_train), batch_size = 32, epochs = 70, verbose=2)


for date in test_months:


	df_test["transactiondate"] = pd.to_datetime(date)  # placeholder value for preliminary version
	df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
	df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
	df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
	df_test["transactiondate"] = df_test["transactiondate"].dt.day     
	x_test = df_test[train_columns]

	print('Shape of x_test:', x_test.shape)
	print("Preparing x_test...")
	for c in x_test.dtypes[x_test.dtypes == object].index.values:
	    x_test[c] = (x_test[c] == True)
	  
	print("\nPredicting with neural network model...")
	#print("x_test.shape:",x_test.shape)
	y_pred_ann = nn.predict(x_test)

	print( "\nPreparing results for write..." )
	nn_pred = y_pred_ann.flatten()
	print( "Type of nn_pred is ", type(nn_pred) )
	print( "Shape of nn_pred is ", nn_pred.shape )

	print( "\nNeural Network predictions:" )
	print( pd.DataFrame(nn_pred).head() )




# In[6]:


from datetime import datetime

for date in test_months:
    sample[date] = nn_pred

sample["ParcelId"] = sample["parcelid"]
sample.drop(["parcelid"], axis=1, inplace=True)

print( "\nWriting results to disk ..." )
sample.to_csv('NN_sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print( "\nFinished ...")

# Cleanup
del train_data
del prop
del sample
del x_train
del x_test
del df_train
del df_test
del y_pred_ann
gc.collect()


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libnvidia-fatbinaryloader.so.375.66: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [11]:
rec = pd.read_csv('./data/RecAreas_CSV.csv')

In [5]:
rec.keys()

Index([u'KEYWORDS', u'LASTUPDATEDDATE', u'ORGRECAREAID', u'RECAREAEMAIL',
       u'RECAREAFEEDESCRIPTION', u'RECAREAID', u'RECAREALATITUDE',
       u'RECAREALONGITUDE', u'RECAREAMAPURL', u'RECAREANAME', u'RECAREAPHONE',
       u'RECAREARESERVATIONURL', u'STAYLIMIT'],
      dtype='object')

In [7]:
rec.shape

(3809, 13)

In [8]:
print rec[u'RECAREALATITUDE'].isnull().sum(), rec[u'RECAREALONGITUDE'].isnull().sum()

542 539


In [12]:
rec = rec[np.isfinite(rec[u'RECAREALATITUDE']) & np.isfinite(rec[u'RECAREALONGITUDE'])]
rec.shape

(3267, 13)

In [16]:
rec['RECAREALONGITUDE'] = [round(long,2) for long in rec['RECAREALONGITUDE']]
rec['RECAREALATITUDE'] = [round(lat,2) for lat in rec['RECAREALATITUDE']]


In [24]:
dummy = rec.groupby(['RECAREALONGITUDE','RECAREALATITUDE']).count()

In [32]:
dummy

,,KEYWORDS,LASTUPDATEDDATE,ORGRECAREAID,RECAREAEMAIL,RECAREAFEEDESCRIPTION,RECAREAID,RECAREAMAPURL,RECAREANAME,RECAREAPHONE,RECAREARESERVATIONURL,STAYLIMIT
RECAREALONGITUDE,RECAREALATITUDE,,,,,,,,,,,
-177.38,28.21,0,1,0,1,0,1,0,1,1,0,0
-176.62,0.81,0,1,0,1,0,1,0,1,1,0,0
-176.28,0.20,0,1,0,1,0,1,0,1,1,0,0
-172.65,60.38,0,1,0,1,0,1,1,1,1,0,0
-170.70,14.28,0,1,0,1,0,1,0,1,1,0,0
-169.85,-14.23,1,1,1,1,0,1,0,1,1,0,0
-169.53,16.73,0,1,0,1,0,1,0,1,1,0,0
-168.15,14.52,0,1,0,1,0,1,0,1,1,0,0
-168.04,53.93,0,1,0,1,0,1,1,1,1,0,0


In [86]:
pub_schools = pd.read_csv('./data/pubschls.csv')

In [87]:
# pub_schools.keys()

In [88]:
pub_schools = pub_schools[(pub_schools['County'] == 'Los Angeles') | (pub_schools['County'] == 'Orange') | (pub_schools['County'] == 'Ventura')]
pub_schools = pub_schools[pub_schools['StatusType'] != 'Closed']
pub_schools = pub_schools[np.isfinite(pub_schools['Latitude'])]
pub_schools = pub_schools[np.isfinite(pub_schools['Longitude'])]
# pub_schools['Latitude'] = [round(long,2) for long in pub_schools['Latitude']]
# pub_schools['Longitude'] = [round(lat,2) for lat in pub_schools['Longitude']]


# pub_schools['SOCType'].value_counts()
pub_schools = pub_schools[pub_schools['SOC'].isin([60,62,66,65])]

# pub_schools['SOCType'].value_counts()

In [89]:
decimal_places = 2
# print(len(pub_schools))


zones = {}

# for row in pub_schools.iterrows():
# #     print(row[1]['Latitude'])
#     dot_index = str(row[1]['Longitude']).index(".")
#     long_str = str(row[1]['Longitude'])[:dot_index+decimal_places]
#     dot_index = str(row[1]['Latitude']).index(".")
#     lat_str = str(row[1]['Latitude'])[:dot_index+decimal_places]
# #     lat_str = row[1]['Latitude']
# #     long_str = row[1]['Longitude']
#     lat_str = float(lat_str)
#     long_str = abs(float(long_str))
#     if (lat_str,long_str) in zones:
#        zones[(lat_str,long_str)] += 1
#     else:
#        zones[(lat_str,long_str)] = 1


for row in pub_schools.iterrows():
#     print(row[1]['Latitude'])
   dot_index = str(row[1]['Longitude']).index(".")
   long_str = str(row[1]['Longitude'])[:dot_index+decimal_places]
   dot_index = str(row[1]['Latitude']).index(".")
   lat_str = str(row[1]['Latitude'])[:dot_index+decimal_places]
   if (lat_str,long_str) in zones:
       zones[(lat_str,long_str)] += 1
   else:
       zones[(lat_str,long_str)] = 1

In [92]:
print zones

{('33.6', '-117.9'): 34, ('34.5', '-118.1'): 10, ('34.2', '-118.3'): 24, ('34.0', '-118.3'): 80, ('33.9', '-118.2'): 173, ('33.7', '-118.1'): 33, ('34.4', '-118.1'): 9, ('34.1', '-118.2'): 44, ('34.3', '-119.3'): 1, ('34.2', '-119.2'): 21, ('34.2', '-118.4'): 75, ('33.6', '-118.0'): 4, ('34.0', '-118.0'): 57, ('34.1', '-118.8'): 13, ('34.3', '-118.9'): 1, ('33.5', '-117.8'): 1, ('33.7', '-117.8'): 81, ('33.8', '-118.0'): 69, ('34.6', '-118.0'): 6, ('34.3', '-119.0'): 9, ('34.7', '-118.8'): 1, ('34.2', '-118.9'): 4, ('34.1', '-119.2'): 4, ('34.0', '-118.5'): 5, ('34.5', '-117.9'): 5, ('33.9', '-118.4'): 30, ('33.5', '-117.7'): 14, ('34.2', '-119.1'): 23, ('33.5', '-117.6'): 25, ('34.1', '-117.8'): 21, ('33.8', '-117.9'): 60, ('33.9', '-118.3'): 98, ('34.1', '-118.6'): 16, ('34.5', '-118.0'): 19, ('33.6', '-117.5'): 5, ('34.7', '-118.2'): 1, ('33.7', '-118.2'): 24, ('34.6', '-117.8'): 2, ('34.4', '-118.2'): 1, ('34.3', '-118.5'): 12, ('34.0', '-117.2'): 1, ('34.1', '-118.1'): 51, ('34.4'

In [110]:
data = pd.merge(prop_data,train_data,on='parcelid')

In [111]:
data = train[distance_cols]

In [112]:
data = data[np.isfinite(data['latitude'])]
data = data[np.isfinite(data['longitude'])]
decimal_places=2

num_schools = []

for d in data:
    print d

actual_area
poolcnt
latitude
longitude
unitcnt
lotsizesquarefeet
bedroomcnt
calculatedbathnbr
hashottuborspa
fireplacecnt
taxvaluedollarcnt
buildingqualitytypeid
garagecarcnt
age
taxamount
airconditioningtypeid
regionidcounty


In [ ]:
for row in data.iterrows():
    long_ = row[1]['longitude']/1000000
    lat =  row[1]['latitude']/1000000
    #     print(str(row[1]['longitude'])[:5])
    dot_index = str(long_).index(".")
    long_str = str(long_)[:dot_index+decimal_places]
    dot_index = str(lat).index(".")
    lat_str = str(lat)[:dot_index+decimal_places]
    #     print(lat_str)
    print lat_str,long_str
    if (lat_str,long_str) in zones:
        num_schools.append(zones[(lat_str,long_str)])
    else:
        num_schools.append(0)


    

In [116]:
# zones

{('33.1', '-117.2'): 2,
 ('33.3', '-117.5'): 1,
 ('33.3', '-118.3'): 1,
 ('33.4', '-117.5'): 4,
 ('33.4', '-117.6'): 14,
 ('33.4', '-117.7'): 2,
 ('33.5', '-117.5'): 1,
 ('33.5', '-117.6'): 25,
 ('33.5', '-117.7'): 14,
 ('33.5', '-117.8'): 1,
 ('33.6', '-117.5'): 5,
 ('33.6', '-117.6'): 25,
 ('33.6', '-117.7'): 18,
 ('33.6', '-117.8'): 21,
 ('33.6', '-117.9'): 34,
 ('33.6', '-118.0'): 4,
 ('33.7', '-117.7'): 24,
 ('33.7', '-117.8'): 81,
 ('33.7', '-117.9'): 85,
 ('33.7', '-118.0'): 34,
 ('33.7', '-118.1'): 33,
 ('33.7', '-118.2'): 24,
 ('33.7', '-118.3'): 18,
 ('33.7', '-118.4'): 5,
 ('33.8', '-117.7'): 14,
 ('33.8', '-117.8'): 45,
 ('33.8', '-117.9'): 60,
 ('33.8', '-118.0'): 69,
 ('33.8', '-118.1'): 59,
 ('33.8', '-118.2'): 48,
 ('33.8', '-118.3'): 82,
 ('33.8', '-118.4'): 3,
 ('33.9', '-117.8'): 23,
 ('33.9', '-117.9'): 37,
 ('33.9', '-118.0'): 64,
 ('33.9', '-118.1'): 77,
 ('33.9', '-118.2'): 173,
 ('33.9', '-118.3'): 98,
 ('33.9', '-118.4'): 30,
 ('34.0', '-117.2'): 1,
 ('34.0', '